In [ ]:
import PyPDF2
import re
from pprint import pprint
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

current_year = datetime.now().year
print(current_year)

def parse_date(date_string):
    date_format = "%Y-%m-%d"
    try:
        parsed_date = datetime.strptime(date_string, date_format).date()
        return parsed_date
    except ValueError:
        return None


In [ ]:
file_path = "..\\..\\Files\\WaitingList\\"

# Skip from here if not parsing pdf report

* Print with the second option for printing style
* Select all categories - not only "läkarbesök".

In [ ]:
# Open the PDF file in read binary mode
with open(file_path+'Report_after_transfering_2024-12-11_limit_2025-12-31.pdf', 'rb') as file:
    # Create a PDF reader object
    reader = PyPDF2.PdfReader(file)
    
    # Initialize an empty string to store the text
    text = ''
    
    # Iterate over each page in the PDF
    for page in reader.pages:
        # Extract the text from the page and append it to the string
        text += page.extract_text()
    
# Print the extracted text
print(text[:100])


In [ ]:
# Split into pages by splitting on "Utskriftsdatum:"

pages = text.split("Utskriftsdatum:")

In [ ]:
def clean_page(page):
    page = re.sub(r'^[\n\w\W]+LUNDVE Hälsocentralen S:t Hans i Lund', "", page)
    page = re.sub(r'\d{4}-\d{2}-\d{2}\s*$', "", page)
    return page
#print(clean_page(pages[0]))

In [ ]:
example = """Läkarmottagning870356-7890 Anna, Anka
2021-04-21
2021-09-01 2021-10-01Kategori:
Väntat sedan:
Bokas fr o m: Bokas senast:Kallad:
VE Hälsocentralen Sankt Hans LundHerr Testsson Önskad vårdgivare:"""

In [ ]:
def extract_re_group(match, group=1, default=""):
    if not match or not match.group(group):
        return default
    return match.group(group)

In [ ]:
def extract_patient(patient_text):
    patient_text = patient_text.strip()
    visit_type = extract_re_group(re.search(r'(^.+?)\d{6}', patient_text))
    personnr = extract_re_group(re.search(r'(\d\d\d\d\d\d[+-]\d\d\d\d)', patient_text))
    staff_member = extract_re_group(re.search(r'Sankt Hans Lund(.+) Önskad vårdgivare', patient_text))

    if not staff_member:
        # print("FAILED WITH:", patient_text, "------------")
        return None

    age = -1
    waited_since = None
    book_from = None
    book_not_later_than = None

    year = extract_re_group(re.search(r'^.+?(\d{2})', patient_text))
    if (year):
        year = int(year)
        if year < 22:
            year += 2000
        else:
            year += 1900
        age = current_year - year

    dates = re.findall(r'\d{4}-\d{2}-\d{2}', patient_text)
    if len(dates) == 3:
        waited_since = parse_date(dates[0])
        book_from = parse_date(dates[1])
        book_not_later_than = parse_date(dates[2])
        

    patient = {
        "staff_member": staff_member,
        "visit_type": visit_type,
        "Personnr": personnr,
        "age": age,
        "waited_since": waited_since,
        "book_from": book_from,
        "book_not_later_than": book_not_later_than
    }
    
    return patient

# pprint(extract_patient(example))

In [ ]:
def get_patients(page):
    patient_texts = page.split("Enhet:")
    patients = []

    for patient_text in patient_texts:
        patient = extract_patient(patient_text)
        if (patient):
            patients.append(patient)

    return patients

In [ ]:
all_patients = []

for page in pages:
    cleaned_page = clean_page(page)
    patients = get_patients(cleaned_page)
    all_patients.extend(patients)
    
df = pd.DataFrame(all_patients)
len(df)

In [ ]:
# save df as xlsx
df.to_excel("patients.xlsx")

# Start here if loading

In [ ]:
df = pd.read_excel("patients.xlsx")

# Filter

In [ ]:
visit_types = set(df["visit_type"])
visit_types

In [ ]:
visits = df[(
    (df["visit_type"] != "Telefontid") & 
            (df["visit_type"] != "Lab/provtagning"))]
len(visits)

In [ ]:
# FILTER LAST DATE
last_date = "2025-12-31"

visits = visits.copy()
visits.loc[:, 'book_not_later_than'] = pd.to_datetime(visits['book_not_later_than'])
# Filter rows where "staff_member" is in dr_names and "book_no_later_than" is before today's date
filtered_rows = visits.loc[(visits['book_not_later_than'] <  datetime.strptime(last_date, "%Y-%m-%d"))]
len(filtered_rows)
visits = filtered_rows

In [ ]:
dr_names = []

with open(file_path+"läkare.txt", "r") as f:
    dr_names = f.readlines()
    dr_names = [name.strip() for name in dr_names]

dr_names

In [ ]:
# Analyze single doctor
def analyze_doctor_months(doctor_name, start_date=None, end_date=None):
    doctors_df = visits[visits["staff_member"] == doctor_name]

    if start_date and end_date:
        start_date = parse_date(start_date)
        end_date = parse_date(end_date)
        doctors_df = doctors_df[(doctors_df["book_not_later_than"] >= start_date) & (doctors_df["book_not_later_than"] <= end_date)]

    # Create an empty dictionary to store the counts
    month_counts = {}

    # Iterate over each row in the dataframe
    for index, entry in doctors_df.iterrows():
        # Get the "book_not_later_than" date
        book_date = entry['book_not_later_than']
        
        # Check if the date is not None
        if book_date is not None:
            # Extract the month and year from the date
            month_year = book_date.strftime('%Y-%m')
            
            # Increment the count for the corresponding month
            month_counts[month_year] = month_counts.get(month_year, 0) + 1

    # Print the resulting dictionary
    return month_counts
    

In [ ]:
def analyze_doctor_quarters(doctor_name, start_date=None, end_date=None):
    doctors_df = visits[visits["staff_member"] == doctor_name]

    if start_date and end_date:
        start_date = parse_date(start_date)
        end_date = parse_date(end_date)
        doctors_df = doctors_df[(doctors_df["book_not_later_than"] >= start_date) & (doctors_df["book_not_later_than"] <= end_date)]
    
    # Create an empty dictionary to store the counts
    quarter_counts = {}

    # Iterate over each row in the dataframe
    for index, entry in doctors_df.iterrows():
        # Get the "book_not_later_than" date
        book_date = entry['book_not_later_than']
        
        # Check if the date is not None
        if book_date is not None:
            # Extract the year from the date
            year = book_date.year
            # Determine the quarter
            quarter = (book_date.month - 1) // 3 + 1
            
            # Create the quarter string
            quarter_year = f'{year}-Q{quarter}'
            
            # Increment the count for the corresponding quarter
            quarter_counts[quarter_year] = quarter_counts.get(quarter_year, 0) + 1

    # Return the resulting dictionary
    return quarter_counts

In [ ]:
waiting_lists_quarters = {}
waiting_lists_months = {}
for name in dr_names:
    waiting_lists_quarters[name] = analyze_doctor_quarters(name, "2023-01-01", "2025-12-31")
    waiting_lists_months[name] = analyze_doctor_months(name, "2023-01-01", "2025-12-31")

In [ ]:
total_n = 0
personal = []
for name in dr_names:
    length = len(visits[visits["staff_member"] == name])
    total_n += length
    personal.append({ "name": name, "n": length })
    # print(name, len(visits[visits["staff_member"] == name]))

for i in range(len(personal)):
    personal[i]["percent"] = personal[i]["n"] / total_n * 100

personal_df = pd.DataFrame(personal, columns=["name", "n", "percent"])
personal_df.sort_values(by="n", ascending=False, inplace=True)
personal_df

personal_df.to_excel("Personal.xlsx", index=False)

In [ ]:
def plot_data(data):
    names = list(data.keys())
    months = sorted(set(m for person_data in data.values() for m in person_data.keys()))

    x = np.arange(len(months))  # the label locations
    width = 1 / (len(names) + 1)  # the width of the bars

    fig, ax = plt.subplots(figsize=(19.20, 10.80))  # Set the figure size
    for i, name in enumerate(names):
        counts = [data[name].get(month, 0) for month in months]
        ax.bar(x - width/2 + i*width, counts, width, label=name)

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_xlabel('Month')
    ax.set_ylabel('Counts')
    ax.set_title('Counts by quarter and person')
    ax.set_xticks(x)
    ax.set_xticklabels(months, rotation=90)  # Rotate the x-axis labels
    ax.legend()

    fig.tight_layout()

    plt.show()

plot_data(waiting_lists_quarters)

In [ ]:
def plot_single_person_data(name, data):
    if name not in data:
        print(f"No data for {name}")
        return

    try:
        person_data = data[name]
        months = sorted(person_data.keys())
        first_month = months[0]
        last_month = months[-1]
        month_before_first = (datetime.strptime(first_month, '%Y-%m') - pd.DateOffset(months=1)).strftime('%Y-%m')
        month_after_last = (datetime.strptime(last_month, '%Y-%m') + pd.DateOffset(months=1)).strftime('%Y-%m')
        months = [month_before_first] + months + [month_after_last]
        person_data[month_before_first] = 0
        person_data[month_after_last] = 0
        
        counts = [person_data[month] for month in months]

        fig, ax = plt.subplots(figsize=(19.20, 10.80))  # Set the figure size
        ax.plot(months, counts, label=name)  # Plot the line
        ax.fill_between(months, counts, color='skyblue', alpha=0.4)  # Fill under the line

        # Add a vertical line at the current month
        current_month = datetime.now().strftime('%Y-%m')
        if current_month in months:
            ax.axvline(x=months.index(current_month), color='r')

        # Add some text for labels, title and custom x-axis tick labels, etc.
        ax.set_xlabel('Month', fontsize=30)  # Increase the font size
        ax.set_ylabel('Counts', fontsize=30)  # Increase the font size
        ax.set_title(f'Counts by month for {name}', fontsize=40)  # Increase the font size
        ax.set_xticklabels(months, rotation=90, fontsize=20)  # Increase the font size
        ax.legend(fontsize=20)  # Increase the font size

        fig.tight_layout()

        plt.show()
    except Exception as e:
        print(f"Failed with {name}: {e}")

#plot_single_person_data(dr_names[4], waiting_lists_months)

In [ ]:
for name in dr_names:
    plot_single_person_data(name, waiting_lists_months)

In [ ]:
# visits.head()

In [ ]:
visits['book_not_later_than'] = pd.to_datetime(visits['book_not_later_than'])
# Filter rows where "staff_member" is in dr_names and "book_no_later_than" is before today's date
filtered_rows = visits.loc[(visits['staff_member'].isin(dr_names)) & (visits['book_not_later_than'] < datetime.today())]

len(filtered_rows)

# # Get the number of rows
# num_rows = filtered_rows.shape[0]

# num_rows